In [ ]:
#Importaciones necesarias para que funcione el codigo
import pandas as pd
import pyreadr
import os
import numpy as np
import ast
import json
from datetime import datetime

# Nombre de la carpeta donde antes de ejecutar el script se guardan los datos CaliBRANDO2024.dta
carpeta = "insumos"
if not os.path.exists(carpeta):
    os.makedirs(carpeta)

data = pd.read_csv("insumos/reportTQ.csv", delimiter=";")

In [ ]:
# Columnas a procesar
columnas_json = [
    '785 - Describa los miembros de su familia con los que convive actualmente (Incluyéndose usted):',
    '826 - La empresa', '827 - Fondo de empleados de la empresa',
    '828 - Entidad financiera y/o tarjetas de crédito', '829 - Prestamista/Gota a gota',
    '830 - Icetex', '831 - Familiar o amigo', '832 - Comfandi/Caja de Compensación Familiar',
    '833 - Otro', '834 - Seleccione el nivel de formación de su mayor interés:',
    '851 - ¿Cuántos días realizó actividades físicas intensas? (ej. Levantar pesos pesados, correr, andar rápido o en bicicleta)',
    '899 - ¿Cuál considera que debe ser el rol de los padres en la educación de los hijos? (Seleccione las tres opciones que mejor describan su opinión)',
    '911 - ¿Cómo puede influir una buena educación en las oportunidades futuras de los hijos? (Seleccione las tres opciones que mejor describan su opinión)'
]

# Separar las columnas que contienen datos en formato JSON o listas
for col in columnas_json:
    if col in data.columns:
        try:
            data[col] = data[col].apply(lambda x: json.loads(x) if pd.notna(x) else [])
        except Exception as e:
            print(f"Error procesando columna {col}: {e}")

# Procesar la información de la familia
families_info = data['785 - Describa los miembros de su familia con los que convive actualmente (Incluyéndose usted):']
found_members = {}

for index in range(len(data)):
    person_info = f"{data['766 - Número de documento'][index]} - {data['770 - Nombres:'][index]} {data['771 - Apellidos:'][index]}"
    sex = data['778 - ¿Cómo se reconoce?'][index]
    birth_datestr = data['772 - Fecha de nacimiento:'][index]
    time_start_str = str(data['TimeStart'][index])
    
    if isinstance(birth_datestr, str) and birth_datestr.lower() != 'nan':
        birth_date = datetime.strptime(birth_datestr, "%d/%m/%Y")
    else:
        birth_date = None
    
    if isinstance(time_start_str, str) and time_start_str.lower() != 'nan':
        survey_time_start_date = datetime.strptime(time_start_str, "%d/%m/%Y %H:%M")
    else:
        survey_time_start_date = None
    
    if birth_date and survey_time_start_date:
        person_age = str(survey_time_start_date.year - birth_date.year - ((survey_time_start_date.month, survey_time_start_date.day) < (birth_date.month, birth_date.day)))
    else:
        person_age = None

    expected_sex = 'Masculino' if sex == 'Hombre' else 'Femenino'
    
    person_family = families_info[index]
    
    for member in person_family:
        if member['COM-23'] == expected_sex and member['COM-24'] == person_age and member['COM-27'] == 'Trabajo formal':
            found_members[person_info] = member
            break

print(f"Found: {len(found_members)} de {len(data)}")

# Identificar personas sin coincidencias
found_keys = {key.split(' - ')[0] for key in found_members.keys()}
not_found = data[~data['766 - Número de documento'].astype(str).isin(found_keys)]
not_found.to_excel('not_found_members.xlsx', index=False)

# Mostrar detalles de los miembros de la familia
for index in range(len(data)):
    person_family = families_info[index]
    print(f"Total miembros de familia: {len(person_family)}")
    for member in person_family:
        print(f"{member['COM-20']} - {member['COM-21']}")


In [ ]:
#Importaciones necesarias para que funcione el codigo
import pandas as pd
import pyreadr
import os
import numpy as np
import ast
import json
from datetime import datetime

# Nombre de la carpeta donde antes de ejecutar el script se guardan los datos CaliBRANDO2024.dta
carpeta = "insumos"
if not os.path.exists(carpeta):
    os.makedirs(carpeta)

data = pd.read_csv("insumos/reportTQ.csv", delimiter=";")


families_info = data['785 - Describa los miembros de su familia con los que convive actualmente (Incluyéndose usted):']

found_members = {}

for index in range(len(data)):
    person_info = f"{data['766 - Número de documento'][index]} - {data['770 - Nombres:'][index]} {data['771 - Apellidos:'][index]}"

    sex = data['778 - ¿Cómo se reconoce?'][index]
    birth_datestr = data['772 - Fecha de nacimiento:'][index]
    birth_date = datetime.strptime(birth_datestr, "%d/%m/%Y")
    # today = datetime.today()
    survey_time_start_date = datetime.strptime(data['Time Start'][index], "%Y-%m-%d %H:%M:%S")

    # person_age = str(today.year - birth_date.year - ((today.month, today.day) < (birth_date.month, birth_date.day)))
    person_age = str(survey_time_start_date.year - birth_date.year - ((survey_time_start_date.month, survey_time_start_date.day) < (birth_date.month, birth_date.day)))

    if sex == 'Hombre':
        expected_sex = 'Masculino'
    elif sex == 'Mujer':
        expected_sex = 'Femenino'

    person_family = json.loads(families_info[index])

    for member in person_family:
        member_sex = member['COM-23']
        member_age = member['COM-24']
        member_work = member['COM-27']

        if member_sex == expected_sex and member_age == person_age and member_work == 'Trabajo formal':
            found_members[person_info] = member

            break

print(f"Found: {len(found_members)} de {len(data)}")




found_keys = {key.split(' - ')[0] for key in found_members.keys()}
not_found = data[~data['766 - Número de documento'].astype(str).isin(found_keys)]



not_found.to_excel('not_found_members.xlsx', index=False)



for index in range(len(data)):
    person_family = json.loads(families_info[index])

    print(f"Total mimebros de familia: {len(person_family)}")

    for member in person_family:
        print(f"{member['COM-20']} - {member['COM-21']}")

In [ ]:
import pandas as pd
import os
import json
from datetime import datetime

# Nombre de la carpeta donde se guardan los datos
carpeta = "insumos"
if not os.path.exists(carpeta):
    os.makedirs(carpeta)

# Cargar los datos desde el archivo CSV
data = pd.read_csv("insumos/reportTQ.csv", delimiter=";")

# Columnas a procesar
columns_to_process = [
    "785 - Describa los miembros de su familia con los que convive actualmente (Incluyéndose usted):",
    "826 - La empresa",
    "827 - Fondo de empleados de la empresa",
    "828 - Entidad financiera y/o tarjetas de crédito",
    "829 - Prestamista/Gota a gota",
    "830 - Icetex",
    "831 - Familiar o amigo",
    "832 - Comfandi/Caja de Compensación Familiar",
    "833 - Otro",
    "834 - Seleccione el nivel de formación de su mayor interés:",
    "851 - ¿Cuántos días realizó actividades físicas intensas? (ej. Levantar pesos pesados, correr, andar rápido o en bicicleta)",
    "899 - ¿Cuál considera que debe ser el rol de los padres en la educación de los hijos? (Seleccione las tres opciones que mejor describan su opinión)",
    "911 - ¿Cómo puede influir una buena educación en las oportunidades futuras de los hijos? (Seleccione las tres opciones que mejor describan su opinión)"
]

# Función para expandir columnas JSON o listas
def expand_json_column(df, column):
    expanded_data = []
    max_items = 0  # Para saber cuántas columnas crear
    
    for entry in df[column].astype(str):
        try:
            parsed_entry = json.loads(entry.replace("'", "\"")) if entry.startswith("[") else entry
            if isinstance(parsed_entry, list):
                max_items = max(max_items, len(parsed_entry))
                expanded_data.append(parsed_entry)
            else:
                expanded_data.append([parsed_entry])
        except Exception:
            expanded_data.append([entry])
    
    expanded_columns = {}
    for i in range(max_items):
        expanded_columns[f"{column}_item_{i+1}"] = [entry[i] if i < len(entry) else None for entry in expanded_data]
    
    return pd.DataFrame(expanded_columns)

# Expandir las columnas seleccionadas
expanded_columns = []
for col in columns_to_process:
    if col in data.columns:
        expanded_df = expand_json_column(data, col)
        expanded_columns.append(expanded_df)
    else:
        print(f"Advertencia: La columna '{col}' no está en los datos.")

# Lógica para encontrar miembros de la familia que cumplen criterios
families_info = data['785 - Describa los miembros de su familia con los que convive actualmente (Incluyéndose usted):']
found_members = {}

for index in range(len(data)):
    person_info = f"{data['766 - Número de documento'][index]} - {data['770 - Nombres:'][index]} {data['771 - Apellidos:'][index]}"
    
    sex = data['778 - ¿Cómo se reconoce?'][index]
    birth_datestr = str(data['772 - Fecha de nacimiento:'][index])
    time_start_str = str(data['TimeStart'][index])
    
    if birth_datestr.lower() != 'nan':
        birth_date = datetime.strptime(birth_datestr, "%d/%m/%Y")
    else:
        birth_date = None
    
    if time_start_str.lower() != 'nan':
        survey_time_start_date = datetime.strptime(time_start_str, "%d/%m/%Y %H:%M")
    else:
        survey_time_start_date = None
    
    if birth_date and survey_time_start_date:
        person_age = str(survey_time_start_date.year - birth_date.year - ((survey_time_start_date.month, survey_time_start_date.day) < (birth_date.month, birth_date.day)))
    else:
        person_age = None
    
    expected_sex = 'Masculino' if sex == 'Hombre' else 'Femenino'
    
    raw_family_info = str(families_info[index])
    
    if raw_family_info.lower() != 'nan':
        try:
            person_family = json.loads(raw_family_info)
        except json.JSONDecodeError:
            person_family = []
    else:
        person_family = []
    
    for member in person_family:
        if member.get('COM-23') == expected_sex and member.get('COM-24') == person_age and member.get('COM-27') == 'Trabajo formal':
            found_members[person_info] = member
            break

print(f"Found: {len(found_members)} de {len(data)}")
found_keys = {key.split(' - ')[0] for key in found_members.keys()}
not_found = data[~data['766 - Número de documento'].astype(str).isin(found_keys)]
not_found.to_excel('not_found_members.xlsx', index=False)

# Concatenar los datos procesados
processed_data = pd.concat([data.drop(columns=[col for col in columns_to_process if col in data.columns])] + expanded_columns, axis=1)

# Guardar en un nuevo archivo Excel
output_file = "procesados_reportTQ.xlsx"
processed_data.to_excel(output_file, index=False)

print(f"Proceso completado. Datos guardados en {output_file}")